In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
import sklearn
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer

In [2]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [3]:
import xgboost as xgb

WindowsError: [Error 126] 

In [104]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

In [113]:
# посмотрим, что представляют из себя данные
test.head()

,Num,year,week,shift,item_id,f1,f2,f3,f4,f5,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
0,348622,2015,3,3,20447918,960.0,820.0,1128.0,1801.0,1045.0,...,1510.0,580.0,969.0,1635.0,895.0,2140.0,1182.0,1020.0,1293.0,1290.0
1,348623,2015,3,3,20447902,9086.0,12585.0,11595.0,9685.0,12917.0,...,22055.0,14235.0,21195.0,18280.0,18270.0,15851.0,16920.0,18320.0,24116.0,21307.0
2,348624,2015,3,3,20447732,115087.0,147287.0,176065.0,143105.0,202069.0,...,302165.0,162232.0,221622.0,256605.0,240047.0,236630.0,206697.0,245652.0,286179.0,285904.0
3,348625,2015,3,3,20443951,20900.0,24420.0,27068.0,20460.0,25580.0,...,39055.0,14445.0,22450.0,22093.0,31175.0,23355.0,15358.0,18930.0,29643.0,33970.0
4,348626,2015,3,3,20443944,4430.0,5864.0,3310.0,1853.0,2836.0,...,120.0,130.0,60.0,30.0,50.0,20.0,20.0,30.0,0.0,0.0


In [114]:
print len(test)

2016


In [105]:
frac = 1# fraction of learning examples used for model fitting

# sample the train set if your don't want to deel with all examples
train = train.sample(frac=frac, random_state=42)

X = train.drop(['Num','y'], axis=1)
y = train['y']
print(len(X), len(y))

(72457, 72457)


### Подбор параметров

Попробуем подобрать глобальные параметры алгоритма, такие как количество деревьев, максимальная глубина деревьев и размер шага $\eta$, для этого используем кросс валидацию. Хотя алгоритм градиентного бустинга засчет шага $\eta$ оказывается довольно устойчивым к переобучению, так что я ожидаю, что при увеличении количества деревьев, качество увеличится

In [59]:
# будем минимизировать ошибку SMAPE, так как именно она используется для проверки качества
def SMAPE(preds, y_test):
    preds = np.array(preds)
    y_test = np.array(y_test)
    if len(preds) != len(y_test):
        raise BaseException
    return 200*(abs((preds-y_test)*1.)/(abs(preds)+abs(y_test))).mean()

In [60]:
# посмотрим, по каким годам у нас есть данные
years = np.unique(train['year']) 
print(years)

[2012 2013 2014 2015]


In [106]:
# сформируем выборки по годам
X_subtrains = []
y_subtrains = []

for year in years:
    X_subtrains.append(X[X['year']==year])
    y_subtrains.append(train[train['year']==year]['y'])

In [112]:
# посмотрим, сколько элементов в каждой выборке
for i in range(4):
    print "Количество элементов в выборке за ", years[i], " год ", len(X[X['year']==years[i]])

Количество элементов в выборке за  2012  год  235
Количество элементов в выборке за  2013  год  35337
Количество элементов в выборке за  2014  год  35544
Количество элементов в выборке за  2015  год  1341


Видно, что данных за 2012 и за 2015 год не очень много

In [111]:
X_subtr = []

4


In [ ]:
model = XGBRegressor(n_estimators=100, max_depth=14, learning_rate=0.03, seed=95)

x_sub = X[X['year']<2015]
y_sub = train[train['year']<2015]['y']
print len(y_sub)
model.fit(x_sub, y_sub)
preds = model.predict(X_subtrains[3])

In [108]:
print(len(X_subtrains[3]))
print(len(X_subtrains[2]))
print(len(X_subtrains[1]))
print(len(X_subtrains[0]))
#SMAPE(preds, y_subtrains[3])

1341
35544
35337
235


In [88]:
%%time
print validation(X_subtrains, y_subtrains, 100, 14, 0.03)

49.5572293985


 Для решения задачи прогнозирования спроса будем использовать алгоритм градиентного бустинга из библиотеки xgboost

In [4]:
from xgboost import XGBRegressor

WindowsError: [Error 126] 

In [87]:
# Функция принимает данные, разделённые на фолды, количество деревьев, максимальную глубину, размер шага по вектору 
# антиградиента
# Так же можно подобрать параметры регуляризации lambda и alpha, alpha мы использовать не будем, а вот с lambda возможно
# что-нибудь поделать, если не слишком долго будем подбирать другие параметры, пока установим по умолчанию = 1
# на выходе получаем среднее значение ошибки SMAPE

def validation(X, y, n_trees=100 , depth=5, eta=0.1):
    SMAPE_scores = []
    for i in range(2, len(X_subtrains)):
        X_train = X_subtrains[0]
        y_train = y_subtrains[0]
        for j in range(1,i):
            X_train = X_train.append(X_subtrains[j])
            y_train = y_train.append(y_subtrains[j])
        model = XGBRegressor(max_depth=depth, learning_rate=eta, n_estimators=n_trees, seed=95)
        model.fit(X_train, y_train)

        preds = model.predict(X[i])
        SMAPE_scores.append(SMAPE(preds, y[i]))
        
    SMAPE_scores = np.array(SMAPE_scores)
    return SMAPE_scores.mean()

In [19]:
# теперь попробуем подобрать параметры для нашей модели с помощью кросс валидации
n_trees = [25, 50, 100, 150, 200]
etas = [0.01*i for i in range(1, 11)]
depths = [3, 5, 11, 17]

In [20]:
# измерим ошибку в зависимости от параметров
from tqdm import tqdm

scores = []
for n_tree in tqdm(n_trees):
    for eta in etas:
        for depth in depths:
            score = validation(X_subtrains, y_subtrains, n_tree, depth, eta)
            scores.append([score, n_tree, eta, depth])

100%|██████████████████████████████████████████████████████████████████████████████| 10/10 [3:00:50<00:00, 1550.70s/it]


In [22]:
# отсортируем полученные значения по возрастанию score, и посмот sortedа первые 10 значений
scores = sorted(scores)
print(scores)

[[57.832660222224888, 25, 0.05, 17], [58.085145757247801, 25, 0.15000000000000002, 17], [58.088538334765701, 25, 0.05, 11], [58.367212237622482, 50, 0.05, 17], [58.995606891959277, 25, 0.1, 17], [59.141648513366732, 75, 0.05, 17], [59.615137509520615, 100, 0.05, 17], [59.704210091939025, 25, 0.15000000000000002, 11], [59.791461688649868, 50, 0.05, 11], [59.932945981850537, 125, 0.05, 17], [59.95101521877433, 50, 0.15000000000000002, 17], [59.986639041541515, 75, 0.15000000000000002, 17], [60.070225568796765, 50, 0.1, 17], [60.252775614372979, 100, 0.15000000000000002, 17], [60.267849614109743, 125, 0.15000000000000002, 17], [60.328893056224452, 25, 0.2, 17], [60.342125705058571, 25, 0.1, 11], [60.381414499335108, 150, 0.05, 17], [60.411060288628583, 175, 0.15000000000000002, 17], [60.4430947686241, 150, 0.15000000000000002, 17], [60.44527447567733, 200, 0.15000000000000002, 17], [60.470855153992794, 225, 0.15000000000000002, 17], [60.511911199632955, 250, 0.15000000000000002, 17], [60.

In [37]:
%%time

model = XGBRegressor(n_estimators=100, max_depth=5)
model.fit(X, y)

preds = model.predict(test.drop(['Num'], axis=1))

CPU times: user 32.6 s, sys: 108 ms, total: 32.7 s
Wall time: 5.64 s


In [45]:
my_submission['y'] = preds

In [51]:
# In GBM you can get some negative predictions:
print(my_submission[my_submission['y'] < 0])

In [52]:
my_submission['y'] = my_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [53]:
my_submission.to_csv("my_submission.tsv", sep=',', index=False)

In [31]:
a = [[1,2,3], [0,0,0], [-1,2,4]]
print(sorted(a))

[[-1, 2, 4], [0, 0, 0], [1, 2, 3]]


In [38]:
%%time
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
import sklearn
import numpy as np
model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=43)
model.fit(X, y)

preds = model.predict(test.drop(['Num'], axis=1))
    

CPU times: user 1min 17s, sys: 488 ms, total: 1min 18s
Wall time: 1min 19s


In [ ]:
scorer = make_scorer( score_func=SMAPE)

tsv = TimeSeriesSplit(n_splits=3)
model = XGBRegressor(n_estimators=100, max_depth=14, learning_rate=0.03, seed=95)
print cross_val_score(model, X, y, cv=tsv, scoring=scorer)